# Resource Constrained Project Scheduling Problem (RCPSP)
Consider a project that consists of `n` jobs. Jobs are performed using `m` types of resources (e.g. people, machines, etc.). The number of available units of resource `k` is denoted by `R[k]`. Job `i` has to be processed for `d[i]` time units (duration). During this time, `r[i, k]` units of each resource `k` are occupied. Additionally, precedence constraints are defined among jobs: for each `(i, j) in p_tab`, job `i` must be finished before job `j` is started. We say that `i` is a predecessor of `j`, or that `j` is a successor of `i`.

The goal of RCPSP is to find start and finish times of all jobs such that all jobs are completed, precedence and resource constraints are satisfied, and the project makespan (finish time of the last job) is minimized.

Problem instances can be obtained by calling `(file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)` where `idx` indicates the index of the instance.
* You should start your tests on instances with `idx in sel_instances`. 
* If you can, you should solve all instances with `idx in range(n_instances)`. 

In [1]:
import numpy as np
import pandas as pd
from gurobipy import *
data_dir = r'C:\Users\user\Desktop\MSBA 320\Data/'
def read_rcpsp_data(idx):
    data_list = pd.read_csv(data_dir + 'all.txt')
    assert (idx >= 0) and (idx < data_list.shape[0]), "Index must be in the range"
    file = data_list.at[idx, 'File']
    with open(data_dir + file) as f:
        while True:
            data = f.readline().split()
            if data[0] == "jobs": break
        n = int(data[-1]) # number of jobs
        while True:
            data = f.readline().split()
            if data[-1] == "R": break
        m = int(data[-2]) # number of resources
        while True:
            data = f.readline().split()
            if data[0] == "PRECEDENCE": break
        f.readline() # skip line
        p_tab = [] # list of precedences
        for i in range(n):
            data = f.readline().split()
            assert int(data[0]) == i + 1, "First number should be the job number"
            assert int(data[1]) == 1, "Second number should be 1"
            assert int(data[2]) == len(data) - 3, "Third number should be the number of successors"
            for j in data[3:]: p_tab.append((i, int(j) - 1))
        for i in range(n - 1): assert len([(k, j) for (k, j) in p_tab if k == i]) > 0, "Each job except the last should have successors"
        assert len([(k, j) for (k, j) in p_tab if k == n - 1]) == 0, "Last job should have no successors"
        for i in range(1, n): assert len([(j, k) for (j, k) in p_tab if k == i]) > 0, "Each job except the first should have predecessors"
        assert len([(j, k) for (j, k) in p_tab if k == 0]) == 0, "First job should have no predecessors"
        while True:
            data = f.readline().split()
            if data[0] == "REQUESTS/DURATIONS:": break
        f.readline() # skip line
        f.readline() # skip line
        d = [0 for i in range(n)] # d[i] = duration of job i
        r = np.zeros((n, m), dtype=np.int) # r[i, j] = # of units of resource j needed for job i
        for i in range(n):
            data = f.readline().split()
            assert int(data[0]) == i + 1, "First number should be the job number"
            assert int(data[1]) == 1, "Second number should be 1"
            d[i] = int(data[2]) # job duration
            assert len(data) == m + 3, "The remaining numbers should be resource consumtion"
            r[i,:] = [int(j) for j in data[3:]]
        while True:
            data = f.readline().split()
            if data[0] == "RESOURCEAVAILABILITIES:": break
        f.readline() # skip line
        data = f.readline().split()
        R = [int(j) for j in data]
    return (file, n, m, d, p_tab, r, R)

n_instances = pd.read_csv(data_dir + 'all.txt').shape[0]
print('Number of all test instances:', n_instances)
sel_instances = [0, 4, 120, 127, 445, 480, 600, 982, 1114, 1518]
print('Number of selected test instances:', len(sel_instances))
print('Selected instances:')
print('{:<4} {:20} {:>4} {:>4} {:>6} {:>10}'.format('idx', 'file', 'n', 'm', 'd_sum', 'p_tab_len'))
for idx in sel_instances:
    (file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)
    print("{:<4} {:20} {:>4} {:>4} {:>6} {:>10}".format(idx, file, n, m, sum(d), len(p_tab)))
    
idx = 0
(file, n, m, d, p_tab, r, R) = read_rcpsp_data(idx)
print('\nData for example instance with idx =', idx)
print("file =", file)
print("n =", n)
print("m =", m)
print("d =", d)
print("p_tab =", p_tab)
print("R =", R)
print("r =", r)

Number of all test instances: 1560
Number of selected test instances: 10
Selected instances:
idx  file                    n    m  d_sum  p_tab_len
0    J30/j301_1.sm          32    4    158         48
4    J30/j301_5.sm          32    4    119         48
120  J30/j3013_1.sm         32    4    151         48
127  J30/j3013_8.sm         32    4    182         48
445  J30/j3045_6.sm         32    4    195         68
480  J60/j601_1.sm          62    4    329         93
600  J60/j6013_1.sm         62    4    340         93
982  J120/j1203_3.sm       122    4    648        183
1114 J120/j12016_5.sm      122    4    650        183
1518 J120/j12056_9.sm      122    4    711        257

Data for example instance with idx = 0
file = J30/j301_1.sm
n = 32
m = 4
d = [0, 8, 4, 6, 3, 8, 5, 9, 2, 7, 9, 2, 6, 3, 9, 10, 6, 5, 3, 7, 2, 7, 2, 3, 3, 7, 8, 3, 7, 2, 2, 0]
p_tab = [(0, 1), (0, 2), (0, 3), (1, 5), (1, 10), (1, 14), (2, 6), (2, 7), (2, 12), (3, 4), (3, 8), (3, 9), (4, 19), (5, 29), (6, 26), (7

In [2]:
from numpy import cumsum ##finding upperbound
UB=cumsum(d)

In [3]:
##precdence constraint
import pandas as pd
z=list(range(0,n))
succ=[]
pred=[]
F=[] ## sorted activities as per precdence relationships
F_new=[]
for i,j in p_tab:
        succ.append(j)
list_dif = [i for i in z + succ if i not in succ or i not in z]
F.append(list_dif)
for i,j in p_tab:
    if [i] in F:
        F.append([j])
p_new=[]        
for i in z:
    p=max(loc for loc, val in enumerate(F) if val == [i])
    p_new.append(p)
#print(p)
#print(p_new)
p_new.sort()
for i in p_new:
    F_new.append(F[i])
#print(p_new)
#print(F)  
for i in range(0,n):
    F_new[i]=int(F_new[i][0])
dict((el,'') for el in F_new)


{0: '',
 1: '',
 2: '',
 3: '',
 5: '',
 10: '',
 14: '',
 6: '',
 7: '',
 12: '',
 4: '',
 8: '',
 9: '',
 11: '',
 18: '',
 26: '',
 15: '',
 25: '',
 13: '',
 17: '',
 16: '',
 20: '',
 19: '',
 21: '',
 28: '',
 24: '',
 22: '',
 23: '',
 29: '',
 27: '',
 30: '',
 31: ''}

In [15]:
##resource constraint
model=Model('RCPSP')
S=model.addVars(F_new,vtype=GRB.INTEGER,ub=UB)
size=len(S)
L=[]
for i,j in p_tab:
    model.addConstr(S[j]-S[i]>=d[i])
#for i in range(0,size-1):
    #L.append(S[i]+d[i])
o=S
d = dict(zip(F_new, o))
from collections import defaultdict

#second_new = defaultdict(list)

for a,b in d.items():
    second_new[b].append(a)      
for t in second_new:
    v=second_new[t]
    for j in range(len(v)):
        lhs=LinExpr()
    for k in range(m):
        lhs+=r[j][k]
    model.addConstr(lhs <= R[k] )

model.setObjective(S[size-1],GRB.MINIMIZE)
model.optimize()
model.X
print(second_new)
###########################################################END HERE#############################################################

Optimize a model with 80 rows, 32 columns and 96 nonzeros
Variable types: 0 continuous, 32 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+00, 2e+02]
  RHS range        [2e+00, 1e+01]
Found heuristic solution: objective 38.0000000
Presolve removed 80 rows and 32 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 38 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.800000000000e+01, best bound 3.800000000000e+01, gap 0.0000%
defaultdict(<class 'list'>, {0: [0, 0, 0], 1: [1, 1, 1], 2: [2, 2, 2], 3: [3, 3, 3], 5: [5, 5, 5], 10: [10, 10, 10], 14: [14, 14, 14], 6: [6, 6, 6], 7: [7, 7, 7], 12: [12, 12, 12], 4: [4, 4, 4], 8: [8, 8, 8], 9: [9, 9, 9], 11: [11, 11, 11], 18: [18, 18, 18], 26: [26, 26, 26], 15: [15, 15, 15], 25: [25, 25, 25], 13: [13, 13, 13], 17

In [5]:
#print(list(S)+list(d))
t1=[]
for t in S:
    t1.append(t)
print(t1)

[0, 1, 2, 3, 5, 10, 14, 6, 7, 12, 4, 8, 9, 11, 18, 26, 15, 25, 13, 17, 16, 20, 19, 21, 28, 24, 22, 23, 29, 27, 30, 31]


In [396]:
print(S.keys())

[0, 1, 2, 3, 5, 10, 14, 6, 7, 12, 4, 8, 9, 11, 18, 26, 15, 25, 13, 17, 16, 20, 19, 21, 28, 24, 22, 23, 29, 27, 30, 31]


In [397]:
type(d)

list

In [398]:
pd.DataFrame(S)

ValueError: If using all scalar values, you must pass an index

In [9]:
o=model.X

In [10]:
print(o[1])

IndexError: list index out of range

In [7]:
for i in range(0,n):
    o[i]=int(o[i])

IndexError: list index out of range

In [8]:
print(o)

[]


In [403]:
print(F_new)

[0, 1, 2, 3, 5, 10, 14, 6, 7, 12, 4, 8, 9, 11, 18, 26, 15, 25, 13, 17, 16, 20, 19, 21, 28, 24, 22, 23, 29, 27, 30, 31]


In [404]:
for i in range(n):
    F1=dict((el,o[i]) for el in F_new)

In [407]:
d = dict(zip(F_new, o))

In [437]:
from collections import defaultdict

second_new = defaultdict(list)

for a,b in d.items():
    second_new[b].append(a)


second_new[0]

[0, 1, 2, 3]

In [427]:
d.items()

dict_items([(0, 0), (1, 0), (2, 0), (3, 0), (5, 8), (10, 8), (14, 8), (6, 4), (7, 4), (12, 4), (4, 6), (8, 6), (9, 6), (11, 13), (18, 13), (26, 13), (15, 13), (25, 17), (13, 15), (17, 10), (16, 18), (20, 23), (19, 17), (21, 24), (28, 16), (24, 24), (22, 31), (23, 33), (29, 36), (27, 25), (30, 28), (31, 38)])

In [429]:
A.items()

dict_items([('E2', {'7', '5'}), ('E3', {'8', '4'}), ('E5', {'7', '5'}), ('E8', {'8', '4'})])

defaultdict(<class 'list'>, {0: [0, 1, 2, 3], 8: [5, 10, 14], 4: [6, 7, 12], 6: [4, 8, 9], 13: [11, 18, 26, 15], 17: [25, 19], 15: [13], 10: [17], 18: [16], 23: [20], 24: [21, 24], 16: [28], 31: [22], 33: [23], 36: [29], 25: [27], 28: [30], 38: [31]})


In [468]:
print(second_new)
    

defaultdict(<class 'list'>, {0: [0, 1, 2, 3], 8: [5, 10, 14], 4: [6, 7, 12], 6: [4, 8, 9], 13: [11, 18, 26, 15], 17: [25, 19], 15: [13], 10: [17], 18: [16], 23: [20], 24: [21, 24], 16: [28], 31: [22], 33: [23], 36: [29], 25: [27], 28: [30], 38: [31]})


In [469]:
second_new[0][2]

2

TypeError: cannot unpack non-iterable int object

In [486]:
for t in second_new:
    v=second_new[t]
    for j in range(len(v)):
        print(v[j])

0
1
2
3
5
10
14
6
7
12
4
8
9
11
18
26
15
25
19
13
17
16
20
21
24
28
22
23
29
27
30
31


In [488]:
print(second_new)

defaultdict(<class 'list'>, {0: [0, 1, 2, 3], 8: [5, 10, 14], 4: [6, 7, 12], 6: [4, 8, 9], 13: [11, 18, 26, 15], 17: [25, 19], 15: [13], 10: [17], 18: [16], 23: [20], 24: [21, 24], 16: [28], 31: [22], 33: [23], 36: [29], 25: [27], 28: [30], 38: [31]})
